### Packages

In [25]:
# NumCosmo
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import matplotlib.pyplot as plt
import scipy
import numpy as np

# Necessary functions:

from numpy import sqrt
from numpy import log10
from numpy import sinh
from numpy import log as ln

#Calculus
from scipy.integrate import quad #Integrate
from sympy import diff, Symbol, symbols #Derivative
from scipy.optimize import minimize

c = 3*(10**5)
#c =1

### NumCosmo Settings

In [26]:
Ncm.cfg_init ()
cosmo = Nc.HICosmoDEXcdm() #Cosmology object

### Data

In [27]:
ser = Ncm.Serialize.new(0)
Ncm.cfg_init ()
data = ser.from_file("/home/cinthia/NumCosmo/data/nc_data_snia_diag_legacy.obj")
lenz = data.y.len()

muobs = []
sigma = []
zobs = []
for i in range(lenz):
    mi = data.y.get(i)
    si = data.sigma.get(i)
    zi = data.x.get(i)
    
    muobs.append(mi)
    sigma.append(si)
    zobs.append(zi)

muobs = np.array(muobs)
zobs = np.array(zobs)
sigmaobs = np.array(sigma)

### Likelihood (-2ln L)

In [44]:
def mu(z, theta): #This function calculates the modular distance
    
#Theta
    H0, Omega_m0, Omega_l0  = theta[0], theta[1], theta[2]


#Hubble function
#    E2z = Omega_l0 + Omega_k0*((1 + z) ** 2) + Omega_m0*((1 + z) ** 3) + Omega_r0*((1 + z) ** 4) 
#    Ez = E2z ** 0.5
    

#Comoving distance
    def integ(b):
        int_arg = lambda z: ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3))) ** 0.5)
        result = quad(int_arg, 0, b)
        return result[0]
        
    dc_list = []
    for i in range(lenz):
        d = integ (z[i])              #(* c/H0)   
        dc_list.append(d)
    
    dc = np.array(dc_list)      
    Dc =  dc      #( * H0/c)
    

#Temporal comoving distance
#     if Omega_k0 == 0:
#         Dt = Dc
#     else:
#         Dt = sinh((sqrt(Omega_k0)) * Dc) / sqrt(Omega_k0)
    
    Dt = Dc
    
#Luminosity distance
    Dl = (1 + z) * Dt

#Modular distance
    mu = ( 5 * log10(Dl) ) + 25 + ( 5 * log10(c / H0))
    
    return mu 


def f_2lnL(theta): #This function calculates -2lnL (L is the Likelihood).
    arg = []
    for i in range(lenz):
        argi = ( ( mu(zobs,theta)[i] - muobs[i] ) ** 2 ) / ( sigmaobs[i] ** 2 )
        arg.append(argi)
        
    return sum(arg)  


### Best fit

In [45]:
def fit():
    x0 = [70.0, 0.20, 0.80]
    x0_np = np.array(x0)
    bnds = ((40, None), (0, 1.0), (0, 1.0))
    
    f = minimize(f_2lnL, x0_np, bounds = bnds, method='Nelder-Mead', tol=1e-6)
    
    return f


fit = fit()

print(fit)

 final_simplex: (array([[75.79370717,  0.        ,  1.        ],
       [75.79370641,  0.        ,  1.        ],
       [75.79370738,  0.        ,  1.        ],
       [75.79370625,  0.        ,  1.        ]]), array([201.44963538, 201.44963538, 201.44963538, 201.44963538]))
           fun: 201.4496353787574
       message: 'Optimization terminated successfully.'
          nfev: 188
           nit: 100
        status: 0
       success: True
             x: array([75.79370717,  0.        ,  1.        ])


### Fisher matrix

In [41]:
theta1, theta2, theta3 = symbols('H0 Omega_m0 Omega_l0')

# d²(-2lnL)/dtheta1²







TypeError: can't convert expression to float